In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#圖片像素
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] 

In [2]:
attrs = ["amount", "freq", "avg_interval"]

In [3]:
raw_data = pd.read_csv("2022AIGO_H_LAB測試資料_雲端行動科技.csv")
raw_data.dropna(axis=0, subset=["item_tag"], inplace=True)

In [4]:
data_2020 = raw_data[raw_data["datetime"].str.startswith("2020")]
data_2020.loc[:, "datetime"] = pd.to_datetime(data_2020["datetime"], format="%Y-%m-%d")
data_2020["month"] = data_2020["datetime"].dt.month

C:\Users\Sylvia\anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
<ipython-input-4-bb98c96d1407>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2020["month"] = data_2020["datetime"].dt.month


In [5]:
all_user = (data_2020.user_id.unique())

In [6]:
# df_2020 = data_2020[data_2020["item_tag"] != "優惠活動/折扣/集點"]
df_2020 = data_2020[data_2020["item_tag"] != "無法分類"]
df_2020 = df_2020[df_2020["item_tag"] != "餐飲需求"]

In [7]:
#先不要用avg_interval
df_group_alltime = df_2020.groupby(["user_id", "item_tag"])[["amount", "datetime", "inv_num"]].agg(
    amount = ("amount", "sum"),
    freq = ("inv_num", lambda x : len(x.unique())),
    avg_interval = ("datetime", lambda x : x.sort_values().diff().sum().days / len(x))
)

In [8]:
df_ouput_alltime = pd.DataFrame(index = all_user, columns=[f'{i}_{j}'  for i in df_2020.item_tag.unique() for j in attrs])
def make_output_matrix(x):
    id = x.index.get_level_values("user_id")[0]
    x = x.droplevel((0))
    df_temp = x.stack()
    df_temp.index = [f"{x}_{y}" for x,y in df_temp.index]
    df_temp = df_temp.to_frame("").T
    df_ouput_alltime.loc[id, df_temp.columns] = df_temp.values

df_group_alltime.groupby(level = (0)).apply(make_output_matrix)

""


In [9]:
df_ouput_alltime.loc[:, df_ouput_alltime.columns.str.endswith("avg_interval")] = df_ouput_alltime.loc[:, df_ouput_alltime.columns.str.endswith("avg_interval")].replace([0, np.nan],1000)


In [10]:
df_ouput_alltime.fillna(0, inplace= True)

In [122]:
df_ouput_alltime = df_ouput_alltime.abs()

In [12]:
#MinMaxScaler() 在MinMaxScaler中是給定了一個明確的最大值與最小值。每個特徵中的最小值變成了0，最大值變成了1。數據會縮放到到[0,1]之間。
#fit_transform() fit_transform():fit_transform(partData)是先對partData作fit()的功能，找到該partData的整體統計特性之指標，如平均值、標準差、最大最小值等等(能依據不同目的套用這些指標在不同的轉換(即後面的transform()動作)上，再實行transform(partData)以對partData進行標準化(英文稱為normalization, 主要有兩種方法: min-max normalization or standard deviation normalization)或歸一化等動作.
from sklearn.preprocessing import MinMaxScaler
minmax_sc = MinMaxScaler()
df_ouput_alltime_scaled = minmax_sc.fit_transform(df_ouput_alltime)

In [130]:
#暫時不用avg_time
df_ouput_alltime = df_ouput_alltime[df_ouput_alltime.columns[~df_ouput_alltime.columns.str.endswith('avg_interval')]]

# No scaled

In [143]:
#V=W*H
#alpha : Constant that multiplies the regularization terms. Set it to zero to have no regularization.
#l1_ratio : The regularization mixing parameter, with 0 <= l1_ratio <= 1. For l1_ratio = 0 the penalty is an elementwise L2 penalty (aka Frobenius Norm). For l1_ratio = 1 it is an elementwise L1 penalty. For 0 < l1_ratio < 1, the penalty is a combination of L1 and L2.
'''K= 5,10,20,50 alpha= .4, l1_ratio = .5'''
from sklearn.decomposition import NMF
K = 10
nmf = NMF(n_components= K, beta_loss= 'frobenius', init='random',alpha= .4, l1_ratio = .5) #nndsvd更適用於sparse factorization
W = nmf.fit_transform(df_ouput_alltime)
H = nmf.components_    

In [144]:
df_H = pd.DataFrame(H, columns=df_ouput_alltime.columns)
df_H

,乳製風味飲品_amount,乳製風味飲品_freq,綠茶_amount,綠茶_freq,停車費_amount,停車費_freq,碳酸飲料_amount,碳酸飲料_freq,牙刷_amount,牙刷_freq,...,其他文具圖書_amount,其他文具圖書_freq,點鈔機_amount,點鈔機_freq,投影機_amount,投影機_freq,美甲玩具_amount,美甲玩具_freq,美髮剪_amount,美髮剪_freq
0,0.000000,0.000000,0.000000,0.000000,0.028529,0.000000,0.274201,0.006301,0.069472,0.000414,...,0.000000,0.000000,0.000118,1.575721e-07,0.001775,1.900620e-06,0.003452,0.000013,0.000571,0.000003
1,1.115652,0.037799,0.293586,0.008129,0.342446,0.001466,0.192508,0.007315,0.018435,0.000541,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000,1.195815e-07,0.002083,0.000007,0.000000,0.000000
2,0.000000,0.000000,3.137502,0.048729,0.863474,0.000000,0.000000,0.000000,0.501801,0.003573,...,0.000000,0.000000,0.006535,3.411132e-05,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
3,1.175432,0.026607,3.075620,0.021747,0.925098,0.002816,2.341094,0.011835,1.917924,0.007549,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
4,6.938102,0.079137,8.274781,0.112800,0.216466,0.000000,10.011098,0.094730,0.809867,0.007122,...,0.000000,0.000000,0.004290,2.211686e-05,0.088828,1.030511e-04,0.013206,0.000049,0.001346,0.000008
5,29.517807,0.465516,38.241884,0.768404,13.778913,0.314091,16.478961,0.435505,3.788049,0.033901,...,0.005646,0.000263,0.012605,6.095979e-05,0.134697,1.154741e-04,0.064401,0.000237,0.001950,0.000007
6,2.453769,0.049144,1.328091,0.011760,2.919339,0.028124,0.959279,0.027265,1.282986,0.014046,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
7,6.525267,0.131661,12.768609,0.195264,0.000000,0.000000,11.779419,0.249431,0.806202,0.001436,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
8,2.836711,0.040525,10.413621,0.262592,11.751239,0.310038,5.138253,0.115520,1.865504,0.018842,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.021871,0.000079,0.000000,0.000000
9,10.791905,0.203498,24.752224,0.753269,9.970906,0.061260,10.771716,0.255398,1.317463,0.009648,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.006119,0.000038


In [131]:
for i in range(K):
    display(pd.DataFrame(pd.concat([df_H.loc[i, :].T.nlargest(5), df_H[df_H>0].loc[i, :].T.nsmallest(5)])))

,0
店家名稱_amount,8.415544e+02
優惠活動/折扣/集點_amount,3.736909e+02
其他服飾配件_amount,3.441721e+02
其他料理_amount,9.436890e+01
運動飲料_amount,8.529613e+01
藍芽接收/發射器_freq,5.050824e-07
內接式硬碟_freq,1.808065e-06
保健箱/急救包_freq,3.735929e-06
棒球/壘球_freq,4.056544e-06
碎紙機_freq,4.132634e-06


,1
汽油_amount,3.341954e+02
其他飾品_amount,3.249298e+02
其他料理_amount,2.699646e+02
智慧型手機_amount,2.247171e+02
香菸_amount,1.101811e+02
燕窩_freq,1.485246e-07
簡報筆_freq,4.012282e-06
高跟鞋_freq,4.463432e-06
風水/命理/占星書_freq,4.480317e-06
其他袋子_freq,8.580953e-06


,2
手續/服務/外送費_amount,7.311530e+02
運費_amount,5.686237e+02
筆記型電腦_amount,5.721249e+01
優惠活動/折扣/集點_amount,1.359481e+01
乳清蛋白_amount,7.453641e+00
行李秤/手提秤_freq,2.742388e-08
彩妝刷具_freq,2.038443e-07
保健箱/急救包_freq,3.213935e-07
兒童潤膚_freq,3.258222e-07
電腦記憶體_freq,3.424542e-07


,3
襪子_amount,6.237898e+02
口腔特殊護理/清潔_amount,4.040334e+02
卸妝油/乳/水_amount,4.025044e+02
優惠活動/折扣/集點_amount,3.359812e+02
妝前乳/隔離霜_amount,3.259702e+02
電鑽/電動起子_freq,9.534111e-07
其他肉類罐頭_freq,1.091221e-06
高跟鞋_freq,1.696022e-06
吸塵器周邊_freq,1.830766e-06
電子門鎖_freq,2.259420e-06


,4
電費_amount,1.066385e+03
其他料理_amount,1.415806e+02
現沖咖啡_amount,1.777654e+01
儲值現金/點數_amount,1.483989e+01
水費_amount,1.284062e+01
其他茶具用品_freq,2.172466e-08
棒球/壘球_freq,4.924432e-07
充電電池/充電器_freq,1.213025e-06
交換器_freq,1.527543e-06
其他文具圖書_freq,1.655368e-06


In [132]:
for i in range(K):
    display(pd.DataFrame(pd.concat([df_H.loc[i, df_H.columns.str.endswith("amount")].T.nlargest(5), df_H[df_H>0].loc[i, df_H.columns.str.endswith("amount")].T.nsmallest(5)])))
    display(pd.DataFrame(pd.concat([df_H.loc[i, df_H.columns.str.endswith("freq")].T.nlargest(5), df_H[df_H>0].loc[i, df_H.columns.str.endswith("freq")].T.nsmallest(5)])))
    #display(pd.DataFrame(pd.concat([df_H.loc[i, df_H.columns.str.endswith("interval")].T.nlargest(5), df_H[df_H>0].loc[i, df_H.columns.str.endswith("interval")].T.nsmallest(5)])))

,0
店家名稱_amount,841.554407
優惠活動/折扣/集點_amount,373.690947
其他服飾配件_amount,344.172096
其他料理_amount,94.368898
運動飲料_amount,85.296130
其他肉類罐頭_amount,0.000020
其他文具圖書_amount,0.000158
保健箱/急救包_amount,0.000189
祭祀用品_amount,0.000236
快煮壺/電熱水壺/瓶周邊_amount,0.000430


,0
優惠活動/折扣/集點_freq,1.062536e+00
麵包/漢堡/蛋糕_freq,3.890907e-01
其他料理_freq,3.672264e-01
手續/服務/外送費_freq,3.355354e-01
店家名稱_freq,3.259993e-01
藍芽接收/發射器_freq,5.050824e-07
內接式硬碟_freq,1.808065e-06
保健箱/急救包_freq,3.735929e-06
棒球/壘球_freq,4.056544e-06
碎紙機_freq,4.132634e-06


,1
汽油_amount,334.195408
其他飾品_amount,324.929791
其他料理_amount,269.964648
智慧型手機_amount,224.717129
香菸_amount,110.181099
其他袋子_amount,0.000205
奶精_amount,0.000294
其他肉類罐頭_amount,0.000870
其他文具圖書_amount,0.000930
風水/命理/占星書_amount,0.001601


,1
優惠活動/折扣/集點_freq,2.593499e+00
麵包/漢堡/蛋糕_freq,1.089360e+00
其他料理_freq,8.788508e-01
香菸_freq,8.312081e-01
現沖咖啡_freq,8.092159e-01
燕窩_freq,1.485246e-07
簡報筆_freq,4.012282e-06
高跟鞋_freq,4.463432e-06
風水/命理/占星書_freq,4.480317e-06
其他袋子_freq,8.580953e-06


,2
手續/服務/外送費_amount,731.153040
運費_amount,568.623668
筆記型電腦_amount,57.212490
優惠活動/折扣/集點_amount,13.594814
乳清蛋白_amount,7.453641
其他茶具用品_amount,0.000006
棒球/壘球_amount,0.000007
其他袋子_amount,0.000009
美髮剪_amount,0.000011
其他肉類罐頭_amount,0.000018


,2
優惠活動/折扣/集點_freq,9.760668e-01
運費_freq,8.920808e-01
手續/服務/外送費_freq,7.761150e-01
麵包/漢堡/蛋糕_freq,1.785055e-01
瓶/罐裝多多_freq,6.124336e-02
行李秤/手提秤_freq,2.742388e-08
彩妝刷具_freq,2.038443e-07
保健箱/急救包_freq,3.213935e-07
兒童潤膚_freq,3.258222e-07
電腦記憶體_freq,3.424542e-07


,3
襪子_amount,623.789847
口腔特殊護理/清潔_amount,404.033379
卸妝油/乳/水_amount,402.504406
優惠活動/折扣/集點_amount,335.981198
妝前乳/隔離霜_amount,325.970185
其他袋子_amount,0.000007
其他茶具用品_amount,0.000132
奶精_amount,0.000153
其他肉類罐頭_amount,0.000259
藥盒_amount,0.000272


,3
優惠活動/折扣/集點_freq,1.171643e+00
汽油_freq,5.749580e-01
儲值現金/點數_freq,5.268808e-01
襪子_freq,3.822416e-01
妝前乳/隔離霜_freq,2.047795e-01
電鑽/電動起子_freq,9.534111e-07
其他肉類罐頭_freq,1.091221e-06
高跟鞋_freq,1.696022e-06
吸塵器周邊_freq,1.830766e-06
電子門鎖_freq,2.259420e-06


,4
電費_amount,1066.385089
其他料理_amount,141.580588
現沖咖啡_amount,17.776536
儲值現金/點數_amount,14.839893
水費_amount,12.840622
其他茶具用品_amount,0.000031
其他文具圖書_amount,0.000056
其他美髮周邊_amount,0.000102
棒球/壘球_amount,0.000134
料理夾_amount,0.000161


,4
優惠活動/折扣/集點_freq,4.208598e-01
其他料理_freq,3.855312e-01
手續/服務/外送費_freq,3.326638e-01
現沖咖啡_freq,2.385019e-01
麵包/漢堡/蛋糕_freq,1.334213e-01
其他茶具用品_freq,2.172466e-08
棒球/壘球_freq,4.924432e-07
充電電池/充電器_freq,1.213025e-06
交換器_freq,1.527543e-06
其他文具圖書_freq,1.655368e-06


# Scaled

In [ ]:
K = 5
nmf_sc = NMF(n_components= K, init='random', random_state=0, max_iter = 1000)
W_sc = nmf_sc.fit_transform(df_ouput_alltime_scaled)
H_sc = nmf_sc.components_

In [ ]:
df_H_sc = pd.DataFrame(H_sc, columns=df_ouput_alltime.columns)
df_H_sc

In [ ]:
for i in range(K):
    display(pd.DataFrame(pd.concat([df_H_sc.loc[i, :].T.nlargest(5), df_H_sc[df_H_sc>0].loc[i, :].T.nsmallest(5)])))

In [ ]:
for i in range(K):
    display(pd.DataFrame(pd.concat([df_H_sc.loc[i, df_H_sc.columns.str.endswith("amount")].T.nlargest(5), df_H_sc[df_H_sc>0].loc[i, df_H_sc.columns.str.endswith("amount")].T.nsmallest(5)])))
    display(pd.DataFrame(pd.concat([df_H_sc.loc[i, df_H_sc.columns.str.endswith("freq")].T.nlargest(5), df_H_sc[df_H_sc>0].loc[i, df_H_sc.columns.str.endswith("freq")].T.nsmallest(5)])))
    display(pd.DataFrame(pd.concat([df_H_sc.loc[i, df_H_sc.columns.str.endswith("interval")].T.nlargest(5), df_H_sc[df_H_sc>0].loc[i, df_H_sc.columns.str.endswith("interval")].T.nsmallest(5)])))

In [ ]:
K = 10
nmf_sc = NMF(n_components= 10, init='random', random_state=0, max_iter = 1000)
W_sc = nmf_sc.fit_transform(df_ouput_alltime_scaled)
H_sc = nmf_sc.components_

In [ ]:
df_H_sc = pd.DataFrame(H_sc, columns=df_ouput_alltime.columns)
df_H_sc

In [ ]:
for i in range(K):
    display(pd.DataFrame(pd.concat([df_H_sc.loc[i, :].T.nlargest(5), df_H_sc[df_H_sc>0].loc[i, :].T.nsmallest(5)])))